In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
if not os.path.exists('/data'):
    os.makedirs('data')

In [31]:
!wget -o 'ChestXRay2017.zip' https://data.mendeley.com/datasets/rscbjbr9sj/2/files/f12eaf6d-6023-432f-acc9-80c9d7393433/ChestXRay2017.zip?dl=1

In [9]:
from zipfile import ZipFile
with ZipFile('ChestXRay2017.zip', 'r') as dataobj:
    dataobj.extractall(path='data')

In [29]:
string = 'person1905_bacteria_4801.jpeg'
m = string.split('_')
m[1]

'bacteria'

In [30]:
files = os.listdir('data/chest_xray/train/PNEUMONIA')
train_virus_files = []
train_bacteria_files = []
for file in files:
    if file.split('_')[1] == 'virus':
        train_virus_files.append(file)
    else:
        train_bacteria_files.append(file)
print("Length of Virus Infected Cases are in Training Dataset: ", len(train_virus_files))
print("Length of Bacteria Infected Cases are in Testing Dataset: ", len(train_bacteria_files))

Length of Virus Infected Cases are:  1345
Length of Bacteria Infected Cases are :  2540


In [34]:
string = 'NORMAL2-IM-1205-0001.jpeg'
string.split('-')[0]

'NORMAL2'

In [36]:
normal_files_all = os.listdir('data/chest_xray/train/NORMAL')
train_im_files = []
train_normal_files = []
for file in normal_files_all:
    if file.split('-')[0] == 'IM':
        im_files.append(file)
    else:
        normal_files.append(file)
print("Length of IM Files: ", len(im_files))
print("Length of Normal Files: ", len(normal_files))

Length of IM Files:  597
Length of Normal Files:  753
